#### Bike Station Information
Retrieves static station details from the Capital Bikeshare API and stores them in a MySQL database. The dataset includes station names, unique IDs, geographic coordinates (latitude and longitude), region IDs, and capacity. This information is essential for mapping station locations and analyzing infrastructure distribution.

In [1]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from typing import Optional

def fetch_station_information(url: str) -> Optional[pd.DataFrame]:
    """
    Fetches station information from the API and returns a cleaned Pandas DataFrame.

    Args:
        url (str): The API endpoint URL.

    Returns:
        Optional[pd.DataFrame]: A DataFrame with selected station columns or None if an error occurs.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)

        # Convert response to JSON
        data = response.json()

        # Extract station information
        stations = data['data']['stations']

        # Create DataFrame with selected columns
        df = pd.DataFrame(stations, columns=['name', 'short_name', 'station_id', 
                                             'region_id', 'capacity', 'lat', 'lon'])

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching station information: {e}")
        return None

def save_to_database(df: pd.DataFrame, table_name: str) -> None:
    """
    Saves the given DataFrame to a MySQL database.

    Args:
        df (pd.DataFrame): The DataFrame to save.
        table_name (str): The name of the target database table.

    Returns:
        None
    """
    try:
        # Load database credentials from .env file
        load_dotenv()
        DB_USER = os.getenv("DB_USER")
        DB_PASSWORD = os.getenv("DB_PASSWORD")
        DB_HOST = os.getenv("DB_HOST")
        DB_NAME = os.getenv("DB_NAME")

        # Create SQLAlchemy engine
        engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{quote_plus(DB_PASSWORD)}@{DB_HOST}/{DB_NAME}")

        # Save DataFrame to the database
        df.to_sql(table_name, con=engine, if_exists="replace", index=False)
        print(f"Data successfully saved to table '{table_name}'.")

    except Exception as e:
        print(f"Error saving to database: {e}")

if __name__ == "__main__":
    # Define API URL
    API_URL = "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/station_information.json"

    # Fetch and process data
    station_data = fetch_station_information(API_URL)

    # Save to database if data is available
    if station_data is not None:
        save_to_database(station_data, "stations_info_df")


Data successfully saved to table 'stations_info_df'.
